In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

Read the dataset, parse the transactions, and create a dataframe

In [2]:
with open('CDNOW_master.txt') as f:
    dataset = f.read().split("\n")

In [3]:
records = []
for line in dataset:
    if line == '':
        continue
    row = list(filter(lambda token: token != '', line.split(' ')))
    rec = {}
    rec['customerID'] = row[0]
    rec['purchaseDate'] = datetime.strptime(row[1], '%Y%m%d')
    rec['quantity'] = int(row[2])
    rec['price'] = float(row[3])
    records.append(rec)
transactions_df = pd.DataFrame(records)

In [4]:
transactions_df

,customerID,purchaseDate,quantity,price
0,00001,1997-01-01,1,11.77
1,00002,1997-01-12,1,12.00
2,00002,1997-01-12,5,77.00
3,00003,1997-01-02,2,20.76
4,00003,1997-03-30,2,20.76
...,...,...,...,...
69654,23568,1997-04-05,4,83.74
69655,23568,1997-04-22,1,14.99
69656,23569,1997-03-25,2,25.74
69657,23570,1997-03-25,3,51.12


Next we'll create a new column in our dataframe that contains total dollar value for each transaction. This column will be used later when we calculate customer lifetime value for each cohort.

In [5]:
transactions_df['total'] = transactions_df['price'] * transactions_df['quantity']

In order to compute the CLTV for each cohort we first need to figure out what month each person became a customer.

In [6]:
first_transactions_df = transactions_df.groupby('customerID')['purchaseDate'].min().reset_index()

In [7]:
first_transactions_df['firstTransactionMonth'] = first_transactions_df['purchaseDate'].dt.month

In [8]:
first_transactions_df['firstTransactionMonth'].value_counts()

2    8476
1    7846
3    7248
Name: firstTransactionMonth, dtype: int64

Merge the first transaction month with the transactions dataframe so everything is in one place

In [9]:
transactions_df = pd.merge(transactions_df, first_transactions_df[['customerID','firstTransactionMonth']], on='customerID')

In [10]:
transactions_df

,customerID,purchaseDate,quantity,price,total,firstTransactionMonth
0,00001,1997-01-01,1,11.77,11.77,1
1,00002,1997-01-12,1,12.00,12.00,1
2,00002,1997-01-12,5,77.00,385.00,1
3,00003,1997-01-02,2,20.76,41.52,1
4,00003,1997-03-30,2,20.76,41.52,1
...,...,...,...,...,...,...
69654,23568,1997-04-05,4,83.74,334.96,3
69655,23568,1997-04-22,1,14.99,14.99,3
69656,23569,1997-03-25,2,25.74,51.48,3
69657,23570,1997-03-25,3,51.12,153.36,3


In [11]:
def calculate_cltv(df):
    transactions_per_customer = df.groupby('customerID')['purchaseDate'].count()
    avg_frequency = transactions_per_customer.mean()
    
    minmax_purchase_dates_by_customer = df.groupby('customerID')['purchaseDate'].agg(['min','max'])
    
    customer_lifetimes = minmax_purchase_dates_by_customer.apply(lambda row: (row['max'] - row['min']).days, axis=1)
    avg_lifetime = customer_lifetimes.mean()
    
    avg_order_value = transactions_df['total'].mean()
    
    return avg_frequency * avg_lifetime * avg_order_value

In [12]:
transactions_df.groupby('firstTransactionMonth').apply(calculate_cltv)

firstTransactionMonth
1    72214.401472
2    64254.296091
3    58151.591189
dtype: float64